In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local").setAppName("transformations_actions")
sc = SparkContext(conf=conf)

In [3]:
# 설정을 모두 가져오려면
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.port', '57093'),
 ('spark.app.id', 'local-1635982448647'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', '172.30.1.11'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'transformations_actions'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.startTime', '1635982447367')]

In [4]:
# 컨택스트 사용을 멈추려면
sc.stop()

In [6]:
# 새로운 
sc = SparkContext(conf=conf)

In [8]:
# paralleize함수는 textFile과 같이 RDD를 만드는데 쓰입니다
# parallelize 는 리스트로부터 RDD를 만든다

In [9]:
foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면",  "라면", "우동", "라면"])

In [10]:
foods

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:274

In [11]:
foods.collect()
# collect 액션 입니다
# collect 액션을 통해선 벨류를 가져올 수 있는데
# 리스트를 그대로 가져옵
# 개발환경에서만 디버깅을 하면서 사용하고
# 실제 프로덕션 환경에서는 쓰지 않아야 한다
# 데이터를 모두 가져오기 때문에 
# 너무 낭비가 되고
# 스파크를 쓰는 의미가 없어지게 됩니다.

['짜장면',
 '마라탕',
 '짬뽕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짜장면',
 '라면',
 '우동',
 '라면',
 '치킨',
 '돈까스',
 '회',
 '햄버거',
 '피자']

In [12]:
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1,
             '치킨': 1,
             '돈까스': 1,
             '회': 1,
             '햄버거': 1,
             '피자': 1})

In [17]:
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [19]:
foods.first()

'짜장면'

In [19]:
foods.count()

17

In [21]:
foods.distinct().collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동', '치킨', '돈까스', '회', '햄버거', '피자']

In [26]:
foods.distinct().count()

12

In [13]:
a = lambda x: print(x)

In [14]:
data = sc.parallelize([1,2,3,4,5]).foreach(a)
# 요소들을 꺼내서 하나의 함수를 적용시킨다
# 값을 리턴하지 않는다
# RDD에 연산을 하고 로그를 어딘가 저장해놓을때 유용하다

In [16]:
sc.parallelize([2, 3, 4, 5, 6], 2).top(2)

[6, 5]

In [17]:
type(data)

NoneType

In [ ]:
# transformations 
# 는 RDD 아웃풋
# 프로그램이 transformation을 볼때 바로 새로운 RDD를 만들진 않습니다.
# 새로운 알디디는 프로그램이 액션을 만나 연산을 시작할때 생깁니다.

# 프로그램이 코드를 보다 transformation을 만나면
# 신택스 체크 후 
# 이 연산을 해야겠군 하고 기억을 한다음 넘어갑니다
# transformations 를 여러번 거치게  될 수 있는데
# 순서대로 변환 과정을 기억하면서 하나의 Directed Asyclic Gragh DAG 를 만들게 됩니다.
# 방향이 있는 비순환 그래프
# 말이 좀 어렵게 됐는데
# 방향성을 가지고 무한정 순환하지 않는 그래프라는 뜻입니다.


In [101]:
sc.parallelize([1, 2, 3]).map(lambda x: x+2).collect()

[3, 4, 5]

In [22]:
# narrow vs wide
# 트랜스포메이션

# Narrow 부터 알아보자

In [23]:
sample = [5, 5, 4, 2, 3, 9, 2]
nums = sc.parallelize(sample)

In [24]:
nums

ParallelCollectionRDD[19] at readRDDFromFile at PythonRDD.scala:274

In [25]:
nums.collect()

[5, 5, 4, 2, 3, 9, 2]

In [27]:
nums.map(lambda x: x+2).collect()

[7, 7, 6, 4, 5, 11, 4]

In [28]:
nums.map(lambda x: x * x).collect()

[25, 25, 16, 4, 9, 81, 4]

In [30]:
# flatMap RDD
# flatMap은 리스트 안의 요소를 펼쳐볼때 쓰입니다
# flat 하다는 이름이 그런 특징에서 오게 되고
# 예를들어 이렇게 영화 이름 데이터가 있을때 
# 영화 데이터를 공백을 기준으로 펼쳐볼때 쓸 수 있습니다.
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [31]:
moviesRDD = sc.parallelize(movies)

In [33]:
moviesRDD.collect()

['그린 북',
 '매트릭스',
 '토이 스토리',
 '캐스트 어웨이',
 '포드 V 페라리',
 '보헤미안 랩소디',
 '빽 투 더 퓨처',
 '반지의 제왕',
 '죽은 시인의 사회']

In [34]:
flatMoviesRDD = moviesRDD.flatMap(lambda x: x.split(" "))
flatMoviesRDD.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [35]:
filteredRDD = flatMoviesRDD.filter(lambda x: x != "매트릭스")

In [36]:
filteredRDD.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [37]:
# regular expression, mumber

In [ ]:
# 그 다음엔 집합과 연관된 transforms 입니다
# UNION, intersection, subtract

In [38]:
num1 = sc.parallelize([1,2,3, 4])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

In [41]:
numUnion = num1.union(num2)

In [42]:
numUnion.collect()

[1, 2, 3, 4, 5, 6]

In [ ]:
# with replacement
# [1,2,3]
# [2, 3]
# [1, 2, 3]

In [44]:
numUnion.sample(True, .5).collect()

[1, 1, 3]

In [54]:
numUnion.sample(True, .5, seed=2).collect()

[1, 1, 2, 4, 5, 6]

In [52]:
numUnion.sample(True, .2).collect()

[3, 4]

In [55]:
# WIDE transformations

In [76]:
foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면",  "라면", "우동", "라면", "치킨", "돈까스", "회", "햄버거", "피자"])

In [77]:
foodsGroup = foods.groupBy(lambda x: x[0])

In [78]:
foodsGroup.collect()

[('짜', <pyspark.resultiterable.ResultIterable at 0x7ffce984bd60>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7ffce9852100>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7ffce9852070>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7ffce9852640>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7ffce9852670>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7ffce9852190>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7ffce9852730>),
 ('치', <pyspark.resultiterable.ResultIterable at 0x7ffce9852790>),
 ('돈', <pyspark.resultiterable.ResultIterable at 0x7ffce98527c0>),
 ('회', <pyspark.resultiterable.ResultIterable at 0x7ffce9852820>),
 ('햄', <pyspark.resultiterable.ResultIterable at 0x7ffce9852880>),
 ('피', <pyspark.resultiterable.ResultIterable at 0x7ffce98528e0>)]

In [85]:
result = foodsGroup.collect()

for (k, v) in result:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']
치 ['치킨']
돈 ['돈까스']
회 ['회']
햄 ['햄버거']
피 ['피자']


In [87]:
nums = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

In [90]:
list(nums.groupBy(lambda x: x % 2).collect()[0][1])

[1, 3, 5, 7, 9]

In [91]:
# intersection

In [96]:
num1 = sc.parallelize([1, 1, 2, 3, 4, 5, 6, 6])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

In [97]:
num1.intersection(num2).collect()

[4, 6, 5]

In [98]:
num1.subtract(num2).collect()

[2, 1, 1, 3]

In [100]:
num1.distinct().collect()

[1, 2, 3, 4, 5, 6]